In [70]:
import requests
import json
import datetime
from pprint import pprint


API_URL = "http://dserver:8000/matches"

session = requests.Session()

# Make the user agent look like a browser
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:24.0) Gecko/20140624 Firefox/24.0 Iceweasel/24.7.0'
})


In [87]:
# Import selenium and chrome webdriver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

import time
import random

STEAM_BASE_URL = "https://steamcommunity.com"
LOGIN_REDIRECT_URL = "https://steamcommunity.com/openid/login?openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&openid.mode=checkid_setup&openid.return_to=https%3A%2F%2Fdota2.net%2Flogin%2Findex.php%3Fgetmid%3Dcsgolounge%26login%3D1&openid.realm=https%3A%2F%2Fdota2.net&openid.ns.sreg=http%3A%2F%2Fopenid.net%2Fextensions%2Fsreg%2F1.1&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select"
LOGIN_URL = "https://csgolounge.com/login/"
MY_BETS_URL = "https://csgolounge.com/mybets/"
LOUNGE_URL = "https://csgolounge.com/"

STEAM_LOGIN_BUTTON_ID = "imageLogin"


def login_to_csgolounge(driver):
    # Load Steam home page before injecting Steam cookies
    driver.get(STEAM_BASE_URL)

    # Load cookies from file
    with open("cookies.json", "r") as f:
        cookies = json.load(f)

    # Add cookies to driver
    for cookie in cookies:
        driver.add_cookie(cookie)

    driver.get(LOGIN_REDIRECT_URL)

    # driver.save_screenshot("before_login_button.png")

    # Wait for the login button to appear
    WebDriverWait(driver, 10).until(
        expected_conditions.element_to_be_clickable((By.ID, STEAM_LOGIN_BUTTON_ID))
    )

    # driver.save_screenshot("after_login_button.png")

    # Wait a random amount of time before clicking the login button
    time.sleep(random.randint(1, 6) / 2)

    # Click the login button
    driver.find_element(By.ID, STEAM_LOGIN_BUTTON_ID).click()

    # Wait for the login to complete
    time.sleep(5)

    # Redirect to lounge login endpoint (sometimes this is unnecessary)
    driver.get(LOGIN_URL)
    

def get_lounge_session_cookie(driver):
    if not driver.current_url.startswith(LOUNGE_URL):
        print(f"Driver url: {driver.current_url} \n redirecting to {LOUNGE_URL}")
        driver.get(LOUNGE_URL)

    return {"PHPSESSID": driver.get_cookie("PHPSESSID")["value"]}

def get_lounge_session_token(driver):
    # Token is stored as javascript variable GetSessionToken
    if driver.current_url != LOUNGE_URL:
        driver.get(LOUNGE_URL)

    token = driver.execute_script("return GetSessionToken")
    return token

def get_lounge_session(driver):
    lounge_session = {
        "cookie": get_lounge_session_cookie(driver),
        "token": get_lounge_session_token(driver)
    }
    # driver.save_screenshot("lounge_session.png")
    return lounge_session

def setup_driver():
    # Set up chrome webdriver
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1280,720")
    chrome_options.add_argument("--disable-gpu")

    # Use existing chrome profile
    chrome_user_data_dir = "./chrome_user_data/"
    chrome_options.add_argument(f"user-data-dir={chrome_user_data_dir}")
    chrome_options.add_argument("--profile-directory=Default")

    return webdriver.Chrome(options=chrome_options)



driver = setup_driver()

try:
    # Login to csgolounge
    print("Logging in...", end="")
    login_to_csgolounge(driver)
    print("Done")

    # Get session cookie and token
    print("Getting session...", end="")
    lounge_session = get_lounge_session(driver)
    print(lounge_session)

    # Save cookies for next time
    driver.get(STEAM_BASE_URL)
    cookies = driver.get_cookies()
    with open("cookies.json", "w+") as f:
        json.dump(cookies, f)

    driver.quit()
except Exception as e:
    driver.save_screenshot("error.png")
    driver.quit()
    raise e



Logging in...Done
Getting session...{'cookie': {'PHPSESSID': 'n5cucg0g6doliq4ue53iib94a7'}, 'token': 'zS659dFqy6cNTrl325pd6m1'}


In [84]:
matches = session.get(API_URL).json()
# parse str(2024-01-31T06:30:04) into seconds using datetime
def parse_time(time):
    return datetime.datetime.strptime(time, "%Y-%m-%dT%H:%M:%S").timestamp()

matches.sort(key=lambda x: parse_time(x['time']))
# pprint(matches)

id = matches[0]['id']
side = 2
bet_size = 1

print(f"Using match: {matches[0]}")

Using match: {'id': 505349, 'time': '2024-01-30T01:00:47', 'competitors': ['Preasy', 'Entropiq'], 'status': '0', 'existing_value': [60.3231, 26.278], 'lounge_odds': [0.6965627457387954, 0.3034372542612045], 'bovada_odds': [0.6875, 0.3125], 'lounge_multiplier': [1.4356208483980433, 3.295574244615268], 'expected_value': [-0.01301066672634521, 0.02986695144227114], 'last_updated': 1706573523}


In [89]:
from IPython.display import HTML
token = lounge_session['token']
cookies = lounge_session['cookie']


place_endpoint = f"/index/placebet/{id}/{side}/{bet_size}/{token}/"
cancel_endpoint = f"/index/cancelbet/{id}/{token}/"
my_bets_endpoint = "/mybets/"

URL = LOUNGE_URL + cancel_endpoint
response = session.get(URL, cookies=cookies)

print(response.text)

zS659dFqy6cNTrl325pd6m1
{"success":true}


In [ ]:
import urllib

login_redirect_url = "https://steamcommunity.com/openid/login?openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&openid.mode=checkid_setup&openid.return_to=https%3A%2F%2Fdota2.net%2Flogin%2Findex.php%3Fgetmid%3Dcsgolounge%26login%3D1&openid.realm=https%3A%2F%2Fdota2.net&openid.ns.sreg=http%3A%2F%2Fopenid.net%2Fextensions%2Fsreg%2F1.1&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select"

steam_redirect_url = "https://dota2.net/login/index.php?getmid=csgolounge&login=1&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&openid.mode=id_res&openid.op_endpoint=https%3A%2F%2Fsteamcommunity.com%2Fopenid%2Flogin&openid.claimed_id=https%3A%2F%2Fsteamcommunity.com%2Fopenid%2Fid%2F76561198372485576&openid.identity=https%3A%2F%2Fsteamcommunity.com%2Fopenid%2Fid%2F76561198372485576&openid.return_to=https%3A%2F%2Fdota2.net%2Flogin%2Findex.php%3Fgetmid%3Dcsgolounge%26login%3D1&openid.response_nonce=2024-01-29T22%3A05%3A26ZDYSkED4RElLs%2Byk0lblV0Gw7iLk%3D&openid.assoc_handle=1234567890&openid.signed=signed%2Cop_endpoint%2Cclaimed_id%2Cidentity%2Creturn_to%2Cresponse_nonce%2Cassoc_handle&openid.sig=RamQfnVufHA4TRCii8hsh32Ubq0%3D"
parsed = urllib.parse.unquote(steam_redirect_url, encoding='utf-8', errors='replace')

print(parsed)


https://dota2.net/login/index.php?getmid=csgolounge&login=1&openid.ns=http://specs.openid.net/auth/2.0&openid.mode=id_res&openid.op_endpoint=https://steamcommunity.com/openid/login&openid.claimed_id=https://steamcommunity.com/openid/id/76561198372485576&openid.identity=https://steamcommunity.com/openid/id/76561198372485576&openid.return_to=https://dota2.net/login/index.php?getmid=csgolounge&login=1&openid.response_nonce=2024-01-29T22:05:26ZDYSkED4RElLs+yk0lblV0Gw7iLk=&openid.assoc_handle=1234567890&openid.signed=signed,op_endpoint,claimed_id,identity,return_to,response_nonce,assoc_handle&openid.sig=RamQfnVufHA4TRCii8hsh32Ubq0=


In [ ]:
print("https://csgolounge.com/match/505349/".startswith("https://csgolounge.com/"))
print(LOUNGE_URL)

True
https://csgolounge.com/
